## Definition and test of function makeUniqueUgVector

This notebook defines and tests a function that shofts the origin of the unit cell in such a way that the phases of two reference structure factors $U_{g1}$ and $U_{g2}$ are set to predefined values (default = 0). This makes the set of 2D structure factors unique.  

The function makeUniqueUgVector requires the following information:
* Ug: structure factor vector
* beams: Miller indices of the reflections corresponding to the Ug-vector
* beam1: index to the first reference beam in the list of beams (should have high amplitude)
* beam2: index to the second reference beam in the list of beams (should have high amplitude and be ideally orthogonal to beam1)
* phase1: Phase that the 1st beam shall be set to (default = 0)
* phase2: Phase that the 2nd beam shall be set to (default = 0)

The function makeUniqueUgVector returns:
* UgUnique: Structure factor vector with shifted unit cell origin, such that the two reference phases are set to the fixed values
* r0: shift that has been applied to the unit cell origin (in fractional coordinates)

In [1]:
import numpy as np
import numpy.linalg as la

Let's see wether we can read the list of beams:

In [14]:
# beamfile = '_BeamList.txt'
# beams = np.asarray(np.loadtxt(beamfile,skiprows=1, usecols= range(1, 4) ), dtype=int )
# print(beams[:,0:2])

In [2]:
outputName = 'r8_test_shift'
filepath = "/Users/chenjie/pyQEDA/pyQEDA_simulations/" + outputName + "/" + outputName + "_experimental_data" +'/'
beamfilename = '_BeamList.txt'
beamfile = filepath + beamfilename
beams = np.asarray(np.loadtxt(beamfile,skiprows=1, usecols= range(1, 4)  ), dtype=int )

The function makeUniqueUgVector determines the 2D shift that must be applied to the unit cell origin, in order to fix the phase of the two reference beams. This can also be extended to a 3D shift, if higher order Lause zone reflections are included. For for the moment, let's stick with 2D shifts only.

Once the necessary shift of the unit cell origin has been determined, it will be applied to all other reflections

Assuming fractional coordinates in real space and miller indices in reciprocal space, the shift $r_0$ can be determined by solving the following linear set of equations:

$$ 2 \pi \left(g_{1x} r_{0,1} + g_{1y} r_{0,2}\right) = \phi_{1,target} - \phi_1$$
$$ 2 \pi \left(g_{2x} r_{0,2} + g_{2y} r_{0,2}\right) = \phi_{2,target} - \phi_2$$

where $\phi_1$ is the phase of $U_{g1}$ and $\phi_2$ is the phase of $U_{g2}$. $\phi_{1,target}$ and $\phi_{2,target}$ are the target values of the phases for $U_{g1}$ and $U_{g2}$.

In order to make sure that we avoid phase wrapping when computing the difference between phases, we use the following relation for computing the phase difference:

$$ \frac{\exp[i \phi_{g,target}]}{U_g} = \frac{\exp[i \phi_{g,target}]}{\left| U_g \right|\exp[i \phi_{g}]} =  \frac{1}{\left| U_g \right|} \exp[i (\phi_{g,target}-\phi_g)] $$ 

Note that the unit cell origin shift can only be determined, if $\vec{g}_1$ and $\vec{g}_2$ are no colinear.

After obtaining the shift in unit cell origin, it can be applied to all structure factors using the following relation:

$$ U_{g,unique} = U_g \exp\left[2 \pi i \left(g_{x} r_{0,1} + g_{y} r_{0,2}\right)\right] $$


In [12]:
def makeUniqueUgVector(Ug,beams,beam1=1,beam2=8,beam3=9,phase1=0,phase2=0,phase3=0):
    '''The function makeUniqueUgVector requires the following information:
    * Ug:     structure factor vector (1D array of complex numbers)
    * beams:  Miller indices of the reflections corresponding to the Ug-vector
    * beam1:  index to the first reference beam in the list of beams
    * beam2:  index to the second reference beam in the list of beams
    * phase1: Phase that the 1st beam shall be set to (default = 0)
    * phase2: Phase that the 2nd beam shall be set to (default = 0)

    The function makeUniqueUgVector returns:
    * UgUnique: Structure factor vector with shifted unit cell origin, such that the two reference 
                phases are set to the fixed values
    * r0:       shift that has been applied to the unit cell origin (in fractional coordinates)'''
    
    # Compute r0 by solving above linear equation
    G = 2.0*np.pi*np.array([beams[beam1,0:3], beams[beam2,0:3],beams[beam3,0:3]])
    dPhi1 = np.angle(np.exp(1j*phase1)/Ug[beam1])
    dPhi2 = np.angle(np.exp(1j*phase2)/Ug[beam2])
    dPhi3 = np.angle(np.exp(1j*phase3)/Ug[beam3])
    dPhi = np.array([dPhi1,dPhi2,dPhi3])
    try:
        r0 = np.linalg.solve(G, dPhi)
    except:
        r0 = np.zeros(3)
        UgUnique = Ug
        print("The three beams seem to be colinear - please chose different beams.")

    # Now, let's apply this offset to the unit cell origin to all structure factors in the list. 
    Nbeams = beams.shape[0]
    UgUnique = np.zeros(Nbeams,dtype=complex)
    for j in range(Nbeams):
        dPhi1 = 2*np.pi*(np.dot(beams[j,0:3],r0))
        UgUnique[j] = Ug[j]*np.exp(1j*dPhi1)
    
    return UgUnique, r0

In [13]:
Nbeams = beams.shape[0]
# In order to test this function, I will generate a random set of structure factors:
Ug = 2.0*np.random.rand(Nbeams)-1+1j*(2.0*np.random.rand(Nbeams)-1.0)

# Now, let's test the above function:
UgUnique, r0 = makeUniqueUgVector(Ug,beams,8,9,10)
print("Applied shift of unit cell origin (fractional coordinates):",r0[0],r0[1],r0[2])
for j in range(12):
    print("Ug[",j,"]:",np.round(np.abs(Ug[j]),5),np.round(np.angle(Ug[j]),5),
          " => ",np.round(np.abs(UgUnique[j]),5),np.round(np.angle(UgUnique[j]),5))

Applied shift of unit cell origin (fractional coordinates): 0.01673449645725724 0.813121811062891 -0.3220631003871653
Ug[ 0 ]: 0.87707 -1.62553  =>  0.87707 -1.62553
Ug[ 1 ]: 1.01614 -1.19774  =>  1.01614 3.06186
Ug[ 2 ]: 0.37276 -1.36617  =>  0.37276 0.65741
Ug[ 3 ]: 0.6568 0.2183  =>  0.6568 -0.95589
Ug[ 4 ]: 0.8821 0.44411  =>  0.8821 -2.75366
Ug[ 5 ]: 0.66886 3.10204  =>  0.66886 -2.33176
Ug[ 6 ]: 0.9725 3.05416  =>  0.9725 -2.05483
Ug[ 7 ]: 0.22223 -0.13385  =>  0.22223 -0.98324
Ug[ 8 ]: 0.96639 3.08541  =>  0.96639 -0.0
Ug[ 9 ]: 0.51764 -0.10515  =>  0.51764 0.0
Ug[ 10 ]: 0.80452 1.91844  =>  0.80452 0.0
Ug[ 11 ]: 1.01078 -2.29917  =>  1.01078 -0.17044


As you can see in the above list, we have changed the phases of all the structure factors. The structure factors 1 and 8 have now phase = 0, as was the default. The phase of all the other beams has been changed according to the change in unit cell origin.

Important message: The diffraction pattern simulated using UgUnique, i.e. the new set of structure factors, will be identical to the diffraction pattern simulated using the original Ug-vector. Please go and verify this. 

As you can see below, we can also fix the phase of the two reference beams to some other value, e.g. we can set them to be 1.5 rad in both cases:

In [13]:
# Now, let's test the above function:
UgUnique, r0 = makeUniqueUgVector(Ug,beams,1,8,1.5,1.5)
print("Applied shift of unit cell origin (fractional coordinates):",r0[0],r0[1])
for j in range(12):
    print("Ug[",j,"]:",np.round(np.abs(Ug[j]),5),np.round(np.angle(Ug[j]),5),
          " => ",np.round(np.abs(UgUnique[j]),5),np.round(np.angle(UgUnique[j]),5))

The two beams seem to be colinear - please chose different beams.
Applied shift of unit cell origin (fractional coordinates): 0.0 0.0
Ug[ 0 ]: 0.3514 2.92442  =>  0.3514 2.92442
Ug[ 1 ]: 0.88625 -2.49799  =>  0.88625 -2.49799
Ug[ 2 ]: 0.36628 0.9684  =>  0.36628 0.9684
Ug[ 3 ]: 0.08409 -1.73739  =>  0.08409 -1.73739
Ug[ 4 ]: 0.74133 -2.1866  =>  0.74133 -2.1866
Ug[ 5 ]: 0.61989 -1.15314  =>  0.61989 -1.15314
Ug[ 6 ]: 1.07842 2.24178  =>  1.07842 2.24178
Ug[ 7 ]: 0.78274 -0.83283  =>  0.78274 -0.83283
Ug[ 8 ]: 0.63083 1.98408  =>  0.63083 1.98408
Ug[ 9 ]: 1.05486 -1.16154  =>  1.05486 -1.16154
Ug[ 10 ]: 0.99275 -3.07955  =>  0.99275 -3.07955
Ug[ 11 ]: 1.01098 -2.58886  =>  1.01098 -2.58886
